In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
from numpy import dot
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import numpy as np
import random
from torch.utils.data import DataLoader
import torch
from gensim.models.keyedvectors import KeyedVectors
import numpy as np

In [82]:
class FastText:
  def __init__(self, word_vectors, a1 , fasttext_vectors,shahname_beyts, weight_type = 'simple', dims=100):
    self.shahname_beyts = shahname_beyts
    self.fasttext_vectors = fasttext_vectors
    self.weight_type = weight_type
    self.docs = a1
    self.dims = dims
    self.word_vectors = word_vectors
    self.vectorizer = TfidfVectorizer(use_idf = True, norm ='l1', ngram_range=(1,1), analyzer='word')
    self.vectorizer.fit_transform(self.docs)
    self.strings = []
    string = ''
    for item in self.vectorizer.get_feature_names():
      string += item
      string += ' '
      self.strings.append(item)
    self.vector = self.vectorizer.transform([string])
    self.vector = self.vector.toarray()
    print('vectorizer sum = ', sum(self.vector))
  

    # print(self.vector)
    # print(self.vector[0])
  

  
  def get_fasttext_average_tf_idf_wights(self,mesra):
    vector_sum = np.zeros(self.dims)
    words = mesra.split(' ')
    try:
      for word in words:
        index = self.strings.index(word)
        vector_sum += self.word_vectors[word] * self.vector[0][index]
    except Exception as e:
      pass
      # print(e.args[0])
    return vector_sum
  
  def get_fastext_average(self,mesra):
    counter = 0
    vector_sum = np.zeros(self.dims)
    words = mesra.split(' ')
    for word in words:
      try:
        vector_sum += self.word_vectors[word]
        counter += 1
      except:
        continue
      vector_sum /= counter
    return vector_sum

  def get_fasttext(self, mesra):
    if self.weight_type == 'tfidf':
      return self.get_fasttext_average_tf_idf_wights(mesra)
    elif self.weight_type == 'simple':
      return self.get_fastext_average(mesra)

  def forward(self,query):
    vector = self.get_fasttext(query)
    vector /= np.linalg.norm(vector)
    final_vector = np.matmul(self.fasttext_vectors, np.transpose(vector))
    sen_v2 = np.copy(self.docs)
    R = np.array(final_vector).argsort()[-10:][::-1]
    new_R =  np.unique(np.floor(R / 2)).astype(int)
    print(R)
    print(new_R)
    ten_tokens = sen_v2[R]
    ten_beyts = self.shahname_beyts[new_R]
    ten_scores = final_vector[R]
    vocab_error = 0
    # ten_tokens = [0] * k
    # ten_scores = [-100] * k
    # vocab_error = 0
    # b  = self.get_fasttext(query)

    # for item in tqdm(self.docs):
    #   try:
    #     c  = self.get_fasttext(item)
    #     if (norm(c)*norm(b)) != 0:
    #       cos_sim = dot(c, b)/(norm(c)*norm(b))
    #       if cos_sim != 0:
    #         for i in range(k):
    #           if ten_scores[i] < cos_sim:
    #             ten_scores[i] = cos_sim
    #             ten_tokens[i] = item
    #             break
    #   except Exception as e:
    #     vocab_error += 1
    #     continue

    # for i in range(len(ten_scores)):
    #   for j in range(len(ten_scores)):
    #     if ten_scores[i] > ten_scores[j]:
    #        ten_scores[i], ten_scores[j] = ten_scores[j], ten_scores[i]
    #        ten_tokens[i], ten_tokens[j] = ten_tokens[j], ten_tokens[i]

    return vocab_error, ten_tokens, ten_scores , ten_beyts



In [ ]:
def get_real_shahname_beyts():
  try:
      # Fix UTF8 output issues on Windows console.
      # Does nothing if package is not installed
      from win_unicode_console import enable
      enable()
  except ImportError:
      pass
  # my_file = open("rahavard_365.txt", "r")
  my_file = open("shahname_data.txt", "r")
    
  # reading the file
  data = my_file.read()
    
  # replacing end of line('/n') with ' ' and
  # splitting the text it further when '.' is seen.
  lst = data.split("\n")

  # printing the data
  my_file.close()
  lst2 = []
  for item in lst:
    x = item.split('****')
    lst2.append(x)
  # beyts = []
  beyts_ = []
  for item in lst2:
    if len(item) == 2:
      beyts_.append(item[0] + '****' + item[1])
  return np.array(beyts_)

In [91]:
# run only once
k = 10 # number of similar beyts to return
word_vectors = KeyedVectors.load_word2vec_format('drive/MyDrive/vec_file.vec')
with open('drive/MyDrive/sen_v2.txt', 'r') as f:
  sen_v2 = f.read()
sen_v2 = sen_v2.split('\n')
sen_v2 = np.array(sen_v2)
fasttext_vectors = np.loadtxt('drive/MyDrive/fasttext_vectors.txt')
nan_indexes = []
for i in range(len(fasttext_vectors)):
  if(np.isnan(fasttext_vectors[i]).any()):
    nan_indexes.append(i)
    if i % 2 == 0:
      nan_indexes.append(i + 1)
    else:
      nan_indexes.append(i - 1)
nan_beyt_indexes = np.unique(np.floor(np.array(nan_indexes)/ 2)).astype(int)
shahname_beyts = get_real_shahname_beyts()
shahname_beyts = np.delete(shahname_beyts, nan_beyt_indexes)
fasttext_vectors= np.delete(fasttext_vectors, nan_indexes, axis = 0)

sen_v2= np.delete(sen_v2, nan_indexes, axis = 0)


In [92]:
fasttext1_2 = FastText(word_vectors, sen_v2,  fasttext_vectors,shahname_beyts, 'simple' )


vectorizer sum =  [5.94364102e-05 2.98065708e-05 3.71215122e-05 ... 6.15497739e-05
 5.17935736e-05 6.15497739e-05]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [93]:
# fasttext_query
words_not_found, mesra, scores, beyts = fasttext1_2.forward('کزین برتر')
beyts

[  104 26964 27747 40986 86978 72576 91074 22404 55371 69098]
[   52 11202 13482 13873 20493 27685 34549 36288 43489 45537]


array(['ببالد ندارد جز این نیرویی****نپوید چو پیوندگان هر سویی',
       'من این دانم ار هست پاسخ جزین****فراخست رای جهان آفرین',
       'سگالش چنین بد نوشته جزین****نکرد ایچ یاد از جهان آفرین',
       'یکی اسب دیدم نگونسار زین****ز بیژن نشانی ندارد جزین',
       'تو این اندکی لشکر من مبین****مراجوی با گرز بر پشت زین',
       'چو کژ آفریدش جهان آفرین****تو مشنو سخن زو و کژی مبین',
       'چو دیدی بگویش کزین سوگرای****ز نزدیک ماکن سوی خانه رای',
       'ندانم که دیدار باشد جزین****یک امشب بکوشیم دست پسین',
       'ندانم که دیدار باشد جزین****که داند چنین جز جهان آفرین',
       'بر و یال و کتف سیاوش جزین****نخواهد کمان نیز بر دشت کین'],
      dtype='<U67')